In [25]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
from misc_functions import load_sig_chans, channel_names_to_indices

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

In [26]:
# Initialize the outer dictionary.
subjects_electrodestoROIs_dict = {}

### make subjects rois to electrodes dict. Don't need to run this more than once.

In [27]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071']

for sub in subjects:
    # sub = 'D0059'
    task = 'GlobalLocal'
    output_name = "Response_fixationCrossBase_1sec_mirror"
    events = ["Response"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False


    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    default_dict = gen_labels(good.info)
    
    # Create rawROI_dict for the subject
    rawROI_dict = defaultdict(list)
    for key, value in default_dict.items():
        rawROI_dict[value].append(key)
    rawROI_dict = dict(rawROI_dict)

    # Filter out keys containing "White-Matter"
    filtROI_dict = {key: value for key, value in rawROI_dict.items() if "White-Matter" not in key}

    # Store the dictionaries in the subjects dictionary
    subjects_electrodestoROIs_dict[sub] = {
        'default_dict': dict(default_dict),
        'rawROI_dict': dict(rawROI_dict),
        'filtROI_dict': dict(filtROI_dict)
    }


# Save to a JSON file
filename = 'subjects_electrodestoROIs_dict.json'
with open(filename, 'w') as file:
    json.dump(subjects_electrodestoROIs_dict, file, indent=4)

print(f"Saved subjects_dict to {filename}")

KeyboardInterrupt: 

### load subjects electrodes to rois dict

In [40]:
# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

Loaded data from subjects_electrodestoROIs_dict.json


### load high gamma data so we can do roi analysis on it
once we have more subjects, turn this into a function and loop over all subjects

In [19]:
import os
import mne
import numpy as np


def load_mne_objects(sub, output_name, task, LAB_root=None):
    """
    Load MNE objects for a given subject and output name.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    return {
        'HG_ev1': HG_ev1,
        'HG_base': HG_base,
        'HG_ev1_rescaled': HG_ev1_rescaled,
        'HG_ev1_evoke': HG_ev1_evoke,
        'HG_ev1_evoke_rescaled': HG_ev1_evoke_rescaled
    }

# Example usage
# sub = 'D0057'
# output_name = "Stimulus_i25and75_fixationCrossBase_1sec_mirror"
# task = 'GlobalLocal'
loaded_objects_D0057_i = load_mne_objects('D0057', "Stimulus_i25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0057_c = load_mne_objects('D0057', "Stimulus_c25and75_fixationCrossBase_1sec_mirror", 'GlobalLocal')


# Access the objects
HG_ev1_D0057_i = loaded_objects_D0057_i['HG_ev1']
HG_base_D0057_i = loaded_objects_D0057_i['HG_base']
HG_ev1_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_rescaled']
HG_ev1_evoke_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke_rescaled']

HG_ev1_D0057_c = loaded_objects_D0057_c['HG_ev1']
HG_base_D0057_c = loaded_objects_D0057_c['HG_base']
HG_ev1_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_rescaled']
HG_ev1_evoke_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke_rescaled']


Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
224 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25and75_fixationCrossBase_1sec_mirror_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
     

### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [46]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# List of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071']

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject


Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071


dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup

In [47]:
dlpfc_electrodes_per_subject = {}

for sub in subjects_electrodestoROIs_dict:
    dlpfc = {key: value for key, value in subjects_electrodestoROIs_dict[sub]['filtROI_dict'].items() 
             if any(roi in key for roi in ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"])}

    # Aggregate electrodes into a list for each subject
    dlpfc_electrodes = []
    for electrodes in dlpfc.values():
        dlpfc_electrodes.extend(electrodes)

    dlpfc_electrodes_per_subject[sub] = dlpfc_electrodes
    print(f'For subject {sub}, dlpfc electrodes are: {dlpfc_electrodes}')

# dlpfc_electrodes_per_subject now contains the list of electrodes for each subject

# Assuming sig_chans_per_subject is a dictionary with subjects as keys and lists of significant channels as values

sig_dlpfc_electrodes_per_subject = {}

for sub, dlpfc_electrodes in dlpfc_electrodes_per_subject.items():
    # Retrieve the list of significant channels for the subject
    sig_chans = sig_chans_per_subject.get(sub, [])

    # Find the intersection of DLPFC electrodes and significant channels for the subject
    sig_dlpfc_electrodes = [elec for elec in dlpfc_electrodes if elec in sig_chans]

    # Store the significant DLPFC electrodes for the subject
    sig_dlpfc_electrodes_per_subject[sub] = sig_dlpfc_electrodes

    # Print results for each subject
    print(f"Subject {sub} significant DLPFC electrodes: {sig_dlpfc_electrodes}")



For subject D0057, dlpfc electrodes are: ['RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12']
For subject D0059, dlpfc electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, dlpfc electrodes are: ['LASF10', 'LASF14', 'LASF15', 'LASF16', 'LMSF5', 'LMSF6', 'LMSF12', 'LPSF10', 'LPSF12', 'RAI10', 'RAI11', 'RAI16', 'RAMF11', 'RAMF12', 'RAMF13', 'RMMF13', 'RMMF14', 'RASF15', 'RMSF8', 'RMSF9', 'RMSF10', 'RMSF7', 'RAMF8', 'RAMF9', 'RAMF10', 'RMMF9', 'RMMF10']
For subject D0065, dlpfc electrodes are: ['RASF13', 'RASF14', 'RASF15', 'RMSF11', 'RMSF12', 'RMSF13', 'RMSF14']
For subject D0069, dlpfc electrodes are: []
For subject D0071, dlpfc electrodes are: ['RIA11', 'RIA12', 'RIA14', 'RIA16', 'RIP14', 'RIP15']
Subject D0057 significant DLPFC electrodes: ['RPI14']
Subject D0059 significant DLPFC electrodes: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
Subject D0063 significant DLPFC electrodes: ['LMSF5', 'LPSF12', 'RAMF12', 'RMMF13', 'RMMF14', 'RMMF10']
Subject

In [35]:
# List of channels in mPFC for subject D0057
mPFC_D0057 = ['RPI14', 'RAMF10', 'RAMF11', 'RAMF12']

# Check if each channel in mPFC_D0057 is in sig_chans_D0057
mPFC_D0057_sig_chans = [chan for chan in mPFC_D0057 if chan in sig_chans_D0057]

# Print results
print("Channels in mPFC_D0057 contained in sig_chans_D0057:", mPFC_D0057_sig_chans)


Channels in mPFC_D0057 contained in sig_chans_D0057: ['RPI14']


### make plot of only specific channels, averaged together

In [21]:
import numpy as np

def compute_average_and_sem(evoked_rescaled_data, selected_channels):
    """
    Compute the average and standard error of the mean for selected channels in an Evoked object.

    Parameters:
    - evoked_rescaled_data (mne.Evoked): The Evoked object containing the data.
    - selected_channels (list of str): A list of channel names to include in the computation.

    Returns:
    - average_data (numpy.ndarray): The average data across the selected channels.
    - sem_data (numpy.ndarray): The standard error of the mean for the selected channels.
    """

    # Find the indices of these channels in the Evoked object
    channel_indices = [evoked_rescaled_data.ch_names.index(ch) for ch in selected_channels if ch in evoked_rescaled_data.ch_names]

    # Check if all channels were found
    if len(channel_indices) != len(selected_channels):
        missing_ch = set(selected_channels) - set(evoked_rescaled_data.ch_names)
        raise ValueError(f"Some channels not found in Evoked object: {missing_ch}")

    # Extract data for these channels
    selected_data = evoked_rescaled_data.data[channel_indices, :]

    # Compute the average and standard error
    average_data = selected_data.mean(axis=0)
    sem_data = selected_data.std(axis=0) / np.sqrt(len(channel_indices))

    return average_data, sem_data

# Example usage
average_mPFC_D0057_i, sem_mPFC_D0057_i = compute_average_and_sem(HG_ev1_evoke_rescaled_D0057_i, ['RPI14', 'RAMF10', 'RAMF11', 'RAMF12'])
average_mPFC_D0057_c, sem_mPFC_D0057_c = compute_average_and_sem(HG_ev1_evoke_rescaled_D0057_c, ['RPI14', 'RAMF10', 'RAMF11', 'RAMF12'])


In [24]:


import matplotlib.pyplot as plt

# Use the times from your evoked data
times = HG_ev1_evoke_rescaled_D0057_i.times  # Modify as needed to match your data

plt.figure(figsize=(10, 6))

# Plot average_mPFC_D0057_i with SEM shading
plt.plot(times, average_mPFC_D0057_i, label='Average mPFC Incongruent')
plt.fill_between(times, average_mPFC_D0057_i - sem_mPFC_D0057_i, average_mPFC_D0057_i + sem_mPFC_D0057_i, alpha=0.3)

# Plot average_mPFC_D0057_c with SEM shading
plt.plot(times, average_mPFC_D0057_c, label='Average mPFC Congruent')
plt.fill_between(times, average_mPFC_D0057_c - sem_mPFC_D0057_c, average_mPFC_D0057_c + sem_mPFC_D0057_c, alpha=0.3)

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average mPFC Signal with Standard Error for D0057')
plt.legend()
plt.savefig(save_dir + f'_mPFC_congruency_zscore_D0057.png')
plt.show()


KeyboardInterrupt: 

#### default_dict has electrodes as keys and ROIs as values. rawROI dict has ROIs as keys and electrodes as values. filtROI is rawROI but without white matter electrodes.

do i need this cell still..?

In [ ]:
# sub = 'D0071'
# task = 'GlobalLocal'
# output_name = "Response_fixationCrossBase_1sec_mirror"
# events = ["Response"]
# times = (-1,1.5)
# base_times = [-1,0]
# LAB_root = None
# channels = None
# full_trial_base = False


# if LAB_root is None:
#     HOME = os.path.expanduser("~")
#     if os.name == 'nt':  # windows
#         LAB_root = os.path.join(HOME, "Box", "CoganLab")
#     else:  # mac
#         LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
#                                 "CoganLab")

# layout = get_data(task, root=LAB_root)
# filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
#                     extension='.edf', desc='clean', preload=False)
# save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

# good = crop_empty_data(filt)
# # %%

# print(f"good channels before dropping bads: {len(good.ch_names)}")
# print(f"filt channels before dropping bads: {len(filt.ch_names)}")

# good.info['bads'] = channel_outlier_marker(good, 3, 2)
# print("Bad channels in 'good':", good.info['bads'])

# filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
# good.drop_channels(good.info['bads'])

# print("Bad channels in 'good' after dropping once:", good.info['bads'])

# print(f"good channels after dropping bads: {len(good.ch_names)}")
# print(f"filt channels after dropping bads: {len(filt.ch_names)}")

# good.load_data()

# # If channels is None, use all channels
# if channels is None:
#     channels = good.ch_names
# else:
#     # Validate the provided channels
#     invalid_channels = [ch for ch in channels if ch not in good.ch_names]
#     if invalid_channels:
#         raise ValueError(
#             f"The following channels are not valid: {invalid_channels}")

#     # Use only the specified channels
#     good.pick_channels(channels)

# ch_type = filt.get_channel_types(only_data_chs=True)[0]
# good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# default_dict = gen_labels(good.info)

# # Creating a new dictionary to group keys by their values
# rawROI_dict = defaultdict(list)

# for key, value in default_dict.items():
#     rawROI_dict[value].append(key)

# # Converting the defaultdict back to a regular dictionary for output
# rawROI_dict = dict(rawROI_dict)

# # Display the new dictionary
# print(rawROI_dict)

# # Creating a new dictionary to exclude keys containing "White-Matter"
# filtROI_dict = {key: value for key, value in rawROI_dict.items() if "White-Matter" not in key}

# # Display the new dictionary
# print(filtROI_dict)

In [4]:
for sub in subjects_electrodestoROIs_dict:
    front = {key: value for key, value in subjects_electrodestoROIs_dict[sub]['filtROI_dict'].items() if "front" in key}
    print(f'For subject {sub}, frontal electrodes are: {front}')

For subject D0057, frontal electrodes are: {'ctx_rh_S_front_sup': ['RAI12', 'RAI13'], 'ctx_rh_G_front_sup': ['RAI15', 'RAI16'], 'ctx_rh_G_front_middle': ['RPI14', 'RAMF10', 'RAMF11', 'RAMF12'], 'ctx_rh_G_front_inf-Orbital': ['RAIF11', 'RAIF12'], 'ctx_rh_G_front_inf-Triangul': ['RAIF13', 'RAIF14']}
For subject D0059, frontal electrodes are: {'ctx_lh_S_front_inf': ['LMMF9', 'LMMF11'], 'ctx_lh_G_front_middle': ['LMMF12', 'LPSF16']}
For subject D0063, frontal electrodes are: {'ctx_lh_G_front_sup': ['LASF10', 'LASF14', 'LASF15', 'LASF16', 'LMSF5', 'LMSF6', 'LMSF12', 'LPSF10', 'LPSF12'], 'ctx_rh_G_front_inf-Opercular': ['RAI5'], 'ctx_rh_S_front_inf': ['RAI10', 'RAI11'], 'ctx_rh_G_front_middle': ['RAI16', 'RAMF11', 'RAMF12', 'RAMF13', 'RMMF13', 'RMMF14'], 'ctx_rh_G_front_sup': ['RASF15', 'RMSF8', 'RMSF9', 'RMSF10'], 'ctx_rh_S_front_sup': ['RMSF7'], 'ctx_rh_S_front_middle': ['RAMF8', 'RAMF9', 'RAMF10', 'RMMF9', 'RMMF10']}
For subject D0065, frontal electrodes are: {'ctx_rh_S_front_sup': ['RASF

In [83]:
# front = {key: value for key, value in filtROI_dict.items() if "front" in key}
# print(f'For subject {sub}, frontal electrodes are: {front}')

For subject D0071, frontal electrodes are: {'ctx_rh_S_front_sup': ['RAI12', 'RAI13'], 'ctx_rh_G_front_sup': ['RAI15', 'RAI16'], 'ctx_rh_G_front_middle': ['RPI14', 'RAMF10', 'RAMF11', 'RAMF12'], 'ctx_rh_G_front_inf-Orbital': ['RAIF11', 'RAIF12'], 'ctx_rh_G_front_inf-Triangul': ['RAIF13', 'RAIF14']}


### maybe delete this stuff later? i don't know what it's for..

In [85]:
# sub = 'D0059'
# task = 'GlobalLocal'
# output_name = "Response_fixationCrossBase_1sec_mirror"
# events = ["Response"]
# times = (-1,1.5)
# base_times = [-1,0]
# LAB_root = None
# channels = None
# full_trial_base = False


# if LAB_root is None:
#     HOME = os.path.expanduser("~")
#     if os.name == 'nt':  # windows
#         LAB_root = os.path.join(HOME, "Box", "CoganLab")
#     else:  # mac
#         LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
#                                 "CoganLab")

# layout = get_data(task, root=LAB_root)
# filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
#                     extension='.edf', desc='clean', preload=False)
# save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

# good = crop_empty_data(filt)
# # %%

# print(f"good channels before dropping bads: {len(good.ch_names)}")
# print(f"filt channels before dropping bads: {len(filt.ch_names)}")

# good.info['bads'] = channel_outlier_marker(good, 3, 2)
# print("Bad channels in 'good':", good.info['bads'])

# filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
# good.drop_channels(good.info['bads'])

# print("Bad channels in 'good' after dropping once:", good.info['bads'])

# print(f"good channels after dropping bads: {len(good.ch_names)}")
# print(f"filt channels after dropping bads: {len(filt.ch_names)}")

# good.load_data()

# # If channels is None, use all channels
# if channels is None:
#     channels = good.ch_names
# else:
#     # Validate the provided channels
#     invalid_channels = [ch for ch in channels if ch not in good.ch_names]
#     if invalid_channels:
#         raise ValueError(
#             f"The following channels are not valid: {invalid_channels}")

#     # Use only the specified channels
#     good.pick_channels(channels)

# ch_type = filt.get_channel_types(only_data_chs=True)[0]
# good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# D0059_default_dict = gen_labels(good.info)

# print(D0059_default_dict == default_dict)

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (185) does not match the number of channels in the raw data file (184). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_acq-01_space-ACPC_electrodes.tsv.


...

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (185) does not match the number of channels in the raw data file (184). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_acq-01_space-ACPC_electrodes.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (185) does not match the number of channels in the raw data file (184). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


good channels before dropping bads: 184
filt channels before dropping bads: 184
outlier round 1 channels: ['LMPF15']
outlier round 2 channels: ['LMPF15', 'LASP3']
outlier round 2 channels: ['LMPF15', 'LASP3', 'LASP5']
outlier round 2 channels: ['LMPF15', 'LASP3', 'LASP5', 'LASP6']
outlier round 2 channels: ['LMPF15', 'LASP3', 'LASP5', 'LASP6', 'LMPT2']
outlier round 2 channels: ['LMPF15', 'LASP3', 'LASP5', 'LASP6', 'LMPT2', 'LPT16']
Bad channels in 'good': ['LMPF15', 'LASP3', 'LASP5', 'LASP6', 'LMPT2', 'LPT16']
Bad channels in 'good' after dropping once: []
good channels after dropping bads: 178
filt channels after dropping bads: 178
Reading 0 ... 6582517  =      0.000 ...  3214.120 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
False


In [92]:
# print(D0059_default_dict == default_dict)
# # Creating a new dictionary to group keys by their values
# D0059_rawROI_dict = defaultdict(list)

# for key, value in D0059_default_dict.items():
#     D0059_rawROI_dict[value].append(key)

# # Converting the defaultdict back to a regular dictionary for output
# D0059_rawROI_dict = dict(D0059_rawROI_dict)

# print(D0059_rawROI_dict == rawROI_dict)

# D0059_filtROI_dict = {key: value for key, value in D0059_rawROI_dict.items() if "White-Matter" not in key}
# print(D0059_filtROI_dict == filtROI_dict)
# D0059_front = {key: value for key, value in D0059_filtROI_dict.items() if "front" in key}
# print(D0059_front == front)

False
False
False
{'ctx_lh_S_front_inf': ['LMMF9', 'LMMF11'], 'ctx_lh_G_front_middle': ['LMMF12', 'LPSF16']}
